In [1]:
!pip install timezonefinder

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
     |████████████████████████████████| 51.3 MB 76 kB/s  eta 0:00:011
     |████████████████████████████████| 1.0 MB 40.8 MB/s eta 0:00:01


In [18]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.window import Window
from timezonefinder import TimezoneFinder
import logging
import pandas as pd

In [2]:
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["YARN_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"

In [3]:
# DEV Constants
GEO_DIR = "/user/solovyovyu/geo.csv"
EVENTS_DIR = "/user/solovyovyu/data/geo/events"
OUT_PATH = "/user/solovyovyu/analytics"

# PROD Constants
# EVENTS_DIR = "/user/master/data/geo/events"

In [4]:
def setup_logging():
    """Configure logging"""
    logger = logging.getLogger(__name__)
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(message)s")
    return logger

logger = setup_logging()

In [5]:
def get_distance(lat1, lon1, lat2, lon2):
    R = 6371
    d_lat = F.radians(lat2 - lat1)
    d_lon = F.radians(lon2 - lon1)

    a = F.sin(d_lat / 2) * F.sin(d_lat / 2) + F.cos(F.radians(lat1)) * F.cos(F.radians(lat2)) * F.sin(d_lon / 2) * F.sin(d_lon / 2)
    c = 2 * F.atan2(F.sqrt(a), F.sqrt(1 - a))
    return R * c

get_distance_udf = F.udf(get_distance, DoubleType())

In [6]:
def get_timezone(lat, lng):
    if lat is None or lng is None:
        print("lat or lng is not defined")
        return None
    tf = TimezoneFinder()
    return tf.timezone_at(lat=lat, lng=lng)

get_timezone_udf = F.udf(get_timezone, StringType())

In [7]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Mart User") \
    .getOrCreate()

25/02/08 09:24:24 WARN Utils: Your hostname, fhm2kr3d8qh6mdnbmqj1 resolves to a loopback address: 127.0.1.1; using 172.16.0.22 instead (on interface eth0)
25/02/08 09:24:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/08 09:24:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
geo_df = spark.read.options(delimiter=";", header=True).csv(GEO_DIR) \
    .withColumn("lat", F.regexp_replace("lat", ",", ".").cast(DoubleType())) \
    .withColumn("lng", F.regexp_replace("lng", ",", ".").cast(DoubleType())) \
    .withColumn("timezone", get_timezone_udf(F.col("lat"), F.col("lng"))) \
    .withColumnRenamed("lat", "geo_lat") \
    .withColumnRenamed("lng", "geo_lon")

geo_df.show()

+---+----------+--------+--------+-------------------+
| id|      city| geo_lat| geo_lon|           timezone|
+---+----------+--------+--------+-------------------+
|  1|    Sydney| -33.865|151.2094|   Australia/Sydney|
|  2| Melbourne|-37.8136|144.9631|Australia/Melbourne|
|  3|  Brisbane|-27.4678|153.0281| Australia/Brisbane|
|  4|     Perth|-31.9522|115.8589|    Australia/Perth|
|  5|  Adelaide|-34.9289|138.6011| Australia/Adelaide|
|  6|Gold Coast|-28.0167|   153.4| Australia/Brisbane|
|  7|Cranbourne|-38.0996|145.2834|Australia/Melbourne|
|  8|  Canberra|-35.2931|149.1269|   Australia/Sydney|
|  9| Newcastle|-32.9167|  151.75|   Australia/Sydney|
| 10|Wollongong|-34.4331|150.8831|   Australia/Sydney|
| 11|   Geelong|  -38.15|  144.35|Australia/Melbourne|
| 12|    Hobart|-42.8806| 147.325|   Australia/Hobart|
| 13|Townsville|-19.2564|146.8183| Australia/Brisbane|
| 14|   Ipswich|-27.6167|152.7667| Australia/Brisbane|
| 15|    Cairns|-16.9303|145.7703| Australia/Brisbane|
| 16| Toow

In [9]:
events_df = spark.read.parquet(EVENTS_DIR)
events_df.printSchema()

root
 |-- event: struct (nullable = true)
 |    |-- admins: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- channel_id: long (nullable = true)
 |    |-- datetime: string (nullable = true)
 |    |-- media: struct (nullable = true)
 |    |    |-- media_type: string (nullable = true)
 |    |    |-- src: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- message_channel_to: long (nullable = true)
 |    |-- message_from: long (nullable = true)
 |    |-- message_group: long (nullable = true)
 |    |-- message_id: long (nullable = true)
 |    |-- message_to: long (nullable = true)
 |    |-- message_ts: string (nullable = true)
 |    |-- reaction_from: string (nullable = true)
 |    |-- reaction_type: string (nullable = true)
 |    |-- subscription_channel: long (nullable = true)
 |    |-- subscription_user: string (nullable = true)
 |    |-- tags: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |   

In [ ]:
message_df = events_df \
    .where("event_type == 'message'")\
    .select(
        "event.message_id",
        F.col("event.message_from").alias("user_id"),
        F.coalesce(F.col("event.message_ts"), F.col("event.datetime")).alias("datetime"),
        "lat",
        "lon"
    )
message_df.show()

+----------+-------+--------------------+-------------------+------------------+
|message_id|user_id|            datetime|                lat|               lon|
+----------+-------+--------------------+-------------------+------------------+
|   1127462|  71465| 2022-05-02 16:50:47|-18.578987300310608| 147.7775954571949|
|      3246| 103904|2021-05-02 15:17:...|-31.743744317426316| 116.5087283951571|
|   1091238| 148246| 2022-05-02 17:25:36|-27.051275676890445| 153.5416357389716|
|     27077|  90144|2021-05-02 20:23:...| -37.88798126426612| 144.4811649892654|
|    906925|  69218|2021-05-02 17:41:...| -31.57547940640118|116.33442925090102|
|    715276|  43365|2021-05-02 14:06:...|-35.051272605611516|150.02178291846198|
|    153382| 128844|2021-05-02 08:58:...| -20.91052453577834|149.70859896828964|
|   1033194|  46233|2021-05-02 04:18:...|-16.463811532627744| 146.5570480155715|
|   1094346| 160134| 2022-05-02 14:30:54| -36.46529225232276|144.82588694902455|
|    480191|  27311|2021-05-

In [11]:
message_with_distance = message_df.join(geo_df, how="cross") \
    .withColumn("distance", get_distance(F.col("lat"), F.col("lon"), F.col("geo_lat"), F.col("geo_lon"))) \
    .select(
        "user_id",
        'message_id',
        'distance',
        "datetime",
        'city',
        "timezone"
    )

message_with_distance.show()

+-------+----------+------------------+-------------------+----------+-------------------+
|user_id|message_id|          distance|           datetime|      city|           timezone|
+-------+----------+------------------+-------------------+----------+-------------------+
|  71465|   1127462| 1733.507737785072|2022-05-02 16:50:47|    Sydney|   Australia/Sydney|
|  71465|   1127462|2156.1920118947874|2022-05-02 16:50:47| Melbourne|Australia/Melbourne|
|  71465|   1127462|1124.5930742415435|2022-05-02 16:50:47|  Brisbane| Australia/Brisbane|
|  71465|   1127462|3519.2180511517136|2022-05-02 16:50:47|     Perth|    Australia/Perth|
|  71465|   1127462|2031.0291736671134|2022-05-02 16:50:47|  Adelaide| Australia/Adelaide|
|  71465|   1127462|1195.7441657683428|2022-05-02 16:50:47|Gold Coast| Australia/Brisbane|
|  71465|   1127462|2184.0268170542145|2022-05-02 16:50:47|Cranbourne|Australia/Melbourne|
|  71465|   1127462|1863.2709600479216|2022-05-02 16:50:47|  Canberra|   Australia/Sydney|

In [12]:
window = Window.partitionBy("message_id", "user_id").orderBy("distance")

message_with_city = message_with_distance \
    .withColumn("rank", F.row_number().over(window))\
    .filter(F.col("rank") == 1) \
    .select(
        "message_id",
        "user_id",
        "datetime",
        "city",
        "timezone"
    )

message_with_city.show()

+----------+-------+--------------------+----------+-------------------+
|message_id|user_id|            datetime|      city|           timezone|
+----------+-------+--------------------+----------+-------------------+
|       386| 152383|2021-05-02 11:31:...|  Maitland|   Australia/Sydney|
|       434|  89858|2021-05-03 18:29:...|   Bendigo|Australia/Melbourne|
|       563|   6596|2021-05-03 07:58:...|  Brisbane| Australia/Brisbane|
|       636| 126426|2021-05-02 18:40:...|Townsville| Australia/Brisbane|
|       654| 120346|2021-05-02 08:36:...|     Perth|    Australia/Perth|
|       713|  89858|2021-05-03 17:46:...|   Bendigo|Australia/Melbourne|
|       991|  42878|2021-05-02 21:10:...|  Brisbane| Australia/Brisbane|
|      1217|  43997|2021-05-02 15:48:...|  Adelaide| Australia/Adelaide|
|      1365|  65291|2021-05-03 21:26:...|   Bendigo|Australia/Melbourne|
|      1497| 128641|2021-05-02 07:06:...|  Maitland|   Australia/Sydney|
|      1974| 156683|2021-05-02 19:31:...|    Darwin

In [13]:
window_spec = Window.partitionBy("user_id", "city").orderBy("datetime")

home_city_df = message_with_city \
    .withColumn("prev_city", F.lag("city").over(window_spec)) \
    .withColumn("is_new_city", (F.col("city") != F.col("prev_city")).cast("int")) \
    .withColumn("group_id", F.sum("is_new_city").over(window_spec)) \
    .withColumn("stay_duration", F.datediff(F.lead("datetime").over(window_spec), F.col("datetime"))) \
    .filter(F.col("stay_duration") >= 27) \
    .groupBy("user_id", "city").agg(
        F.max("stay_duration").alias("total_stay")
    ) \
    .orderBy("total_stay", ascending=False) \
    .groupBy("user_id").agg(
        F.first("city").alias("home_city")
    )

In [14]:
mart_user = message_with_city \
    .groupBy("user_id").agg(
        F.last("city").alias("act_city")
    ).join(home_city_df, on="user_id", how="left")

In [15]:
travel_features = message_with_city.groupBy("user_id").agg(
    F.collect_list("city").alias("travel_array"),
    F.count("city").alias("travel_count")
)
mart_user = mart_user.join(travel_features, on="user_id", how="left")

mart_user.show()

+-------+-----------+---------+--------------------+------------+
|user_id|   act_city|home_city|        travel_array|travel_count|
+-------+-----------+---------+--------------------+------------+
|      9|   Canberra|     NULL|          [Canberra]|           1|
|     24|  Newcastle|     NULL|         [Newcastle]|           1|
|    140|     Mackay|     NULL|            [Mackay]|           1|
|    141| Townsville|     NULL|        [Townsville]|           1|
|    211|  Melbourne|     NULL|         [Melbourne]|           1|
|    239|     Hobart|     NULL|            [Hobart]|           1|
|    272|     Darwin|     NULL|[Newcastle, Darwi...|           4|
|    273|      Perth|     NULL|      [Perth, Perth]|           2|
|    279|     Darwin|     NULL|    [Darwin, Darwin]|           2|
|    303|Rockhampton|     NULL|       [Rockhampton]|           1|
|    317|      Perth|     NULL|             [Perth]|           1|
|    442|     Mackay|     NULL|[Mackay, Mackay, ...|           3|
|    487| 

In [16]:
window = Window.partitionBy("user_id").orderBy("datetime")

user_last_message_local_time = message_with_city \
    .withColumn("rank", F.rank().over(window)) \
    .filter(F.col("rank") == 1) \
    .withColumn("local_time", F.from_utc_timestamp(F.col("datetime"), F.col("timezone"))) \
    .select("user_id", "local_time")

In [17]:
# добавить местное время события — время последнего события пользователя, о котором у нас есть данные с учётом таймзоны геопозициии этого события.
mart_user = mart_user.join(user_last_message_local_time, on="user_id", how="left")

mart_user.show()

+-------+-----------+---------+--------------------+------------+--------------------+
|user_id|   act_city|home_city|        travel_array|travel_count|          local_time|
+-------+-----------+---------+--------------------+------------+--------------------+
|      9|   Canberra|     NULL|          [Canberra]|           1|2021-05-03 07:05:...|
|     24|  Newcastle|     NULL|         [Newcastle]|           1|2021-05-04 09:43:...|
|    140|     Mackay|     NULL|            [Mackay]|           1| 2022-05-02 14:50:39|
|    141| Townsville|     NULL|        [Townsville]|           1|2021-05-02 19:41:...|
|    211|  Melbourne|     NULL|         [Melbourne]|           1| 2022-05-03 09:29:30|
|    239|     Hobart|     NULL|            [Hobart]|           1| 2022-05-03 19:14:28|
|    272|     Darwin|     NULL|[Newcastle, Darwi...|           4|2021-05-02 14:27:...|
|    273|      Perth|     NULL|      [Perth, Perth]|           2|2021-05-02 20:46:...|
|    279|     Darwin|     NULL|    [Darwin,